## Single Source Shortest Path
### Overview
* the main focus of this notebook is to solve single source shortest path problem
  + given the starting vertex, find the shortest path to any of the vertices in a weighted graph
  + given the starting vertex, find the shortest paths between the starting vertex and a given target vertex
* edge relexation
  + assuming A is the starting vertex, and the direct distance between A and D is 3
    + if we can find another path from A to D, for example, by A-B-D, which is shorter than 3 (eg. 2), we then say the edge between A and D is relaxed, since we find a shorter path to replace the more "tight" path, which is longer
* two single soruce shortest path algorithms
  + Dijkstra's algorithm
    + can only be used to solve single source shortest path problem in a graph with non-negative weights
  + Bellman-Ford algorithm
    + can solve the single-source shortest path in a weighted directed graph with any weights, including negative weights

### Dijkstra's Algorithm
#### algorithm principles
* question: find the shoretest path from vertex A to all the other vertices in the graph with weighted edges
* process
![image.png](attachment:image.png)
  + using a table with three columns: target, shortest distance from source vertex, and previous vertex
  * round 1, check vertex A, the source vertex, by checking all its edges (AB, AC, AD)
    + for vertex A, the distance is 0, and previous vertex is None
    + for vertex B, edge AB is 1, its distance is 1, previous vertex is A
    + for vertex C, edge AC is 1, its distance is 1, previous vertex is A
    + for vertex D, edge AD is 3, its distance is 3, previous vertex is A
    + for vertex E, no edge from A to E, its distance is infinite, previous vertex is None
    + since all edges from A are traversed, mark Vertex A as visited
  * round 2, from the remaining unvisited vertices, select one with the shortest distance, we select B
    + traverse all edges from vertex B (AB, BD, BE)
    + skip edge AB, since A has been visited
    + for vertex D, BD is 2, and AB is 1, so BD+AB < current distance, skip
    + for vertext E, BE=1, dist(B) = 1, so dist(D) = 2 < infinite, so update dist(D)=2, previous vertex is B
    + mark vertex B as visited
  * round 3, select vertex C, since it has the shortest distance of 1, and traverse all its edges, AC, CD
    + skip vertex A, since it has been visited
    + for vertex D, dist(C)+ edge CD (1) = 2 < current dist, which is 3, update dist(D) = 2, previous vertex is C
    + mark vertex C as visited
  * from remaining unvisited vertices, D and E, we randomly select D, which has AD, CD and DE
    * skip vertices A and C since they are visited
    * for vertex D, DE is 2, dist(D)=2, so dist(D) + DE = 4 > current distance for D, so skip
    * mark vertex D as visited
  * finish vertex E, since all its neighbors are visited
  * we can easily construct the paths from source vertex to other vertices by tracing the previous vertex column
* in general, we take the starting vertex as the center and gradually expand outward while updating the shortest path to reach other vertices
* limitations
  + dijkstra algorithm only works for graphs where all the weights are positive. This is because in each round when we choose the vertex with shortest path, we know it must be the shortest path for that vertex, since paths from other vertices are already at least equal to this path, and any other path from these vertices to this vertex must add extra vertices, which is sure to be longer due to the positive weights of all the edges. However, if the edge weights can be negative, this will not be valid
* Time complexity
  + O(V + ElogV) for binary heap
    + add each vertex to set O(V)
    + in worst case, need to insert all edges to heap, and each time heappop is log(E) = log(V), the same for heappop
* space complexity:
  + O(V) to store V vertices
  + depends on the structure of heap, if can not delete items, will consume an extra O(E) space for heap, otherwise, will only need to store edges of unvisited vertices
    

####  Leetcode 743 Network Delay Time  
* use defaultdict(list) to store the adj_list
  + since it is a directed graph, we only store the entry using the start vertices as key, and append(weight, end_vertex)
* use a list of time_delays to store the shortest distance results for each vertex (remember to offset the index by one)
  + all the vertices have a value of infinite, excep the starting vertex, which has a value of zero
* use d_table to simulate the Dijkstra table to store the intermediate distance results during the process
  + initialize d_table by heappush((0, starting_vertex))
  + we use the heappush and heappop operations on this d_table to pop the shortest distance results for each round
  + if the heappopped distance of a vertex is bigger than the corresponding distance stored in time_delays table, we skip
  + if the heappopped distance equals to the corresponding value in time_delays, this is an updated shortest distance
    + then we find all its neighbouring vertices from the adj_list, and calculate the distances for these vertices, and if they are shorter than the values in time_delays, then 
      + update the time_delays value
      + add the corresponding neighbouring vertices, and their updated shortest distances to heap
* when d_table is empty, check the time_delays table
  + if any of the vertices has distance of infinite, returns -1 (this vertex is not reachable)
  + find the max of the time_delays table and returns
* time complexity
  + O(Elog(V))
  + ignore the time_delays upates which is O(E)
  + assume O(log(E)) = O(log(V))
* space complexity:
  + O(E+V)
  + O(E) for adj_list
  + O(E) for d_table
  + O(V) for time_delays

In [4]:
from heapq import heappop, heappush
from typing import List
class Solution:
    def networkDelayTime(self, times: List[List[int]], n: int, k: int) -> int:
                        
        edges = defaultdict(list)
        for start, end, dist in times:
            edges[start-1].append((dist, end-1))
            
        # can use list for adj_list
        # adj_list = [[] for _ in range(n)]
        # adj_list[start-1].append((dist, end-1))
            
        time_delays = [float("inf")] * n
        time_delays[k-1] = 0
        
        d_table = []
        
        heappush(d_table, (0, k-1))        
        
        while d_table:
            dist, vertex = heappop(d_table)
            
            if dist == time_delays[vertex]:    
            
                for (d, ng) in edges[vertex]:
                    if dist + d < time_delays[ng]:
                        time_delays[ng] = dist + d
                        heappush(d_table, (dist+d, ng))
                    
       
        rs = 0
        for i in range(len(time_delays)):
            if time_delays[i] == float("inf"):
                return -1
            rs = max(rs, time_delays[i])
        return rs     
    